In [37]:
import Pkg
Pkg.add("Cbc")
Pkg.add("JuMP")

  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`


In [50]:
Pkg.add("GLPK")
Pkg.add("Gurobi")
Pkg.add("IterTools")
using JuMP
using Cbc
using DelimitedFiles
using NBInclude
#using GLPK
using Gurobi
const GUROBI_ENV = Gurobi.Env()
const L = 6
using IterTools

# constante epsilon
const ϵ = 0.00001

  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...


Academic license - for non-commercial use only - expires 2021-07-05


No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`


1.0e-5

In [51]:
@nbinclude("dataparser.ipynb");
@nbinclude("Euristic.ipynb");
@nbinclude("classes.ipynb");
@nbinclude("traitement_noeuds.ipynb");
@nbinclude("branch_and_price.ipynb");
@nbinclude("colonnes.ipynb");
#@nbinclude("utiles.ipynb")

  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\33777\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\33777\.julia\environments\v1.5\Manifest.toml`


Academic license - for non-commercial use only - expires 2021-07-05


In [52]:
# Pour chaque paire, on va récupérer les arcs dont ils sont l'origine (calcul du demi-degré extérieur des sommets du graphe)
function degreExt(kep)
    origine = Array{Array{Int64,1},1}()
    for v in 1:nv(kep)
        # On recupère les indices des arcs dont l'origine est v
        origin_v = Array{Int64,1}()
        for (ind,arc) in enumerate(edges(kep))
            if src(arc) == v
                push!(origin_v,ind)
            end
        end
        push!(origine,origin_v)
    end
    return origine
end

# Pour chaque paire, on va récupérer les arcs dont ils sont la destination (calcul du demi-degré intérieur des sommets)
function degreInt(kep)
    dest = Array{Array{Int64,1},1}()
    for v in 1:nv(kep)
        # On recupère les indices des arcs dont l'origine est v
        dest_v = Array{Int64,1}()
        for (ind,arc) in enumerate(edges(kep))
            if dst(arc) == v
                push!(dest_v,ind)
            end
        end
        push!(dest,dest_v)
    end
    return dest
end

#FONCTIONS A AJOUTER

# Recuperation des arcs
function getEdge(kep)
    edge_list = Array{Array{Int64,1},1}()
    for (ind,arc) in enumerate(edges(kep))
            push!(edge_list,[src(arc), dst(arc)])
    end
    return edge_list
end

getEdge(pool)

#Recuperation de tous les chemins de longueur L+1
function getPath_L(pool)
    all_path = simplecycles_hawick_james(pool) #Recuperation de tous les chemins du graphe
    path_L=[]
    L=6
    for i in all_path #Recuperation des chemins de longueur L+1 dans path_L
        if length(i) == L+1
            push!(path_L, i)
        end
    end
    return path_L
end

getPath_L (generic function with 1 method)

In [53]:
# Le probleme

# Le modèle : choisir un des 2 selon votre ORDI
#model = Model()

model=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
set_optimizer_attribute(model, "OutputFlag", 0) #???? parametre a modifier

# Les donnees
path_L = getPath_L(pool)
#Ensemble des arcs sous la forme [i,j]
edge_list = getEdge(pool); #Penser a ajouter la fonction PROBLEME_SANS_REFORMULATION

#Nombre de sommets
E = ne(pool)
V = nv(pool)

#Les variables
@variable(model, x[i in 1:V, j in 1:V], Bin)

#La fonction objective
@objective(model, Max, sum(sum(x[i,j] for i in 1:V) for j in 1:V))

#Les contraintes
for i in 1:V
    for j in 1:V
        if !([i,j] in edge_list)
            @constraint(model, x[i,j]==0)
        end
    end
end

#Deuxieme contrainte
for i in 1:V
    E_i = [] #Ensemble des arcs ou i est l arrivee
    #Recuperer les arcs ou il y a i en extremite
    for e in edge_list
        if e[1] == i
            push!(E_i, e[2])
        end
    end
    #La contrainte sur le sommet i
    @constraint(model, sum(x[i, e] for e in E_i) <= 1)
end

#Troisieme contrainte
for p in path_L #Pour tous les chemins de longueur L+1
    @constraint(model, sum(x[p[j], p[j+1]] for j in 1:L) <= L-1)
end

LoadError: UndefVarError: optimizer_with_attributes not defined

In [54]:
#Résolution du problème
optimize!(model)

LoadError: UndefVarError: optimize! not defined

In [ ]:
#Valeur objective du modèle
objective_value(model)

In [ ]:
#Affichage solution

x_value = JuMP.value.(x)
solu_x = []
for i = 1:V
    for j = 1:V
        if(x_value[i,j]==1)
            push!(solu_x, [i,j])
        end
    end
end